In [3]:
# import packages
import requests
import pandas as pd


Le Global Health Observatory (GHO) est une plateforme de données de l'OMS dédiée à la collecte, l’analyse et la diffusion des données de santé mondiales. 

## Création d'une fonction pour se connecter à l'API de l'OMS, récupérer les jeux de données et créer un dataframe

In [4]:
def get_indicator_data(url, columns=None):
    """
    Récupère et filtre les données d'un indicateur de l'OMS depuis l'API.

    Paramètres :
    - url (str) : URL de l'indicateur dans l'API de l'OMS.
    - columns (list, optionnel) : Liste des colonnes à conserver. Si None, conserve toutes les colonnes.

    Retourne :
    - pd.DataFrame : DataFrame avec les données récupérées et filtrées.
      Retourne None si la requête échoue.
    """
    # Effectuer la requête vers l'API
    response = requests.get(url)
    
    if response.status_code == 200:
        # Charger les données JSON dans un DataFrame
        data = response.json()
        df = pd.DataFrame(data['value'])
        
        # Filtrer les colonnes si une liste est fournie
        if columns:
            existing_columns = [col for col in columns if col in df.columns]
            df_filtered = df[existing_columns]
        else:
            # Conserver toutes les colonnes si 'columns' est None
            df_filtered = df
        
        # Supprimer les lignes entièrement vides (optionnel)
        df_filtered = df_filtered.dropna(how='all')
        
        return df_filtered
    else:
        # Afficher un message d'erreur si la requête échoue
        print("Erreur lors de la récupération des données.")
        return None



In [14]:
columns = ['SpatialDim', 'TimeDim', 'Value', 'NumericValue', 'Low', 'High',"Dim1"]

data_alcohol= get_indicator_data("https://ghoapi.azureedge.net/api/SA_0000001743")


In [15]:
data_alcohol.columns

Index(['Id', 'IndicatorCode', 'SpatialDimType', 'SpatialDim',
       'ParentLocationCode', 'TimeDimType', 'ParentLocation', 'Dim1Type',
       'Dim1', 'TimeDim', 'Dim2Type', 'Dim2', 'Dim3Type', 'Dim3',
       'DataSourceDimType', 'DataSourceDim', 'Value', 'NumericValue', 'Low',
       'High', 'Comments', 'Date', 'TimeDimensionValue', 'TimeDimensionBegin',
       'TimeDimensionEnd'],
      dtype='object')

In [16]:
data_alcohol.head(2)

,Id,IndicatorCode,SpatialDimType,SpatialDim,ParentLocationCode,TimeDimType,ParentLocation,Dim1Type,Dim1,TimeDim,...,DataSourceDim,Value,NumericValue,Low,High,Comments,Date,TimeDimensionValue,TimeDimensionBegin,TimeDimensionEnd
0,7406034,SA_0000001743,COUNTRY,AUT,EUR,YEAR,Europe,SEX,SEX_FMLE,2019,...,None,2.2 [-0.2 - 4.4],2.20072,-0.1707,4.41506,None,2024-08-13T12:03:42.09+02:00,2019,2019-01-01T00:00:00+01:00,2019-12-31T00:00:00+01:00
1,8354381,SA_0000001743,COUNTRY,CPV,AFR,YEAR,Africa,SEX,SEX_MLE,2019,...,None,13.2 [10.2 - 16.1],13.23164,10.2412,16.06331,None,2024-08-13T12:03:42.09+02:00,2019,2019-01-01T00:00:00+01:00,2019-12-31T00:00:00+01:00


In [17]:
# Renommer les colonnes pour qu'elles soient plus compréhensibles
noms_colonnes = {
    'SpatialDim': 'pays',
    'TimeDim': 'annee',
    'Dim1': 'sexe',
    'NumericValue': 'pourcentage_deces'
}

# Appliquer la sélection et le renommage sur le DataFrame
data_alcohol = data_alcohol[columns].rename(columns=noms_colonnes)



In [18]:
# Afficher un aperçu des données
data_alcohol.head()

,pays,annee,Value,pourcentage_deces,Low,High,sexe
0,AUT,2019,2.2 [-0.2 - 4.4],2.20072,-0.17070,4.41506,SEX_FMLE
1,CPV,2019,13.2 [10.2 - 16.1],13.23164,10.24120,16.06331,SEX_MLE
2,BOL,2019,2.4 [1.5 - 3.6],2.37865,1.46098,3.60384,SEX_FMLE
3,ROU,2019,8.8 [6.7 - 11.1],8.76505,6.67160,11.09415,SEX_BTSX
4,CRI,2019,6.0 [4.1 - 8.2],6.01584,4.07295,8.20691,SEX_MLE


Création de la fonction pour appeler data_alcohol

In [7]:
def load_data_alcohol():
    """
    Charge et prépare les données sur les décès attribuables à l'alcool.
    """
    # URL de l'API pour les décès liés à l'alcool
    api_url = "https://ghoapi.azureedge.net/api/SA_0000001743"

    # Colonnes à extraire
    columns = ['SpatialDim', 'TimeDim', 'Value', 'NumericValue', 'Low', 'High',"Dim1"]

    # Charger les données depuis l'API
    data_alcohol = get_indicator_data(api_url, columns)

    # Renommer les colonnes pour les rendre plus compréhensibles
    noms_colonnes = {
        'SpatialDim': 'pays',
        'TimeDim': 'annee',
        'Value': 'valeur',
        'NumericValue': 'pourcentage_deces',
        'Low': 'borne_inferieure',
        'High': 'borne_superieure',
        "Dim1":"sexe"
    }
    data_alcohol = data_alcohol.rename(columns=noms_colonnes)

    # Nettoyer ou transformer les données si nécessaire (par exemple, retirer les valeurs nulles)
    data_alcohol = data_alcohol.dropna(subset=['pourcentage_deces'])

    return data_alcohol

In [31]:
data_alcohol=load_data_alcohol()

In [32]:
data_alcohol["sexe"].unique()

array(['SEX_FMLE', 'SEX_MLE', 'SEX_BTSX'], dtype=object)

In [ ]:
def load_metabolic_risk_data(indicator_code, countries=None, years=None, sexes=None):
    """
    Charge les données de risque métabolique depuis l'API GHO de l'OMS.

    Paramètres :
    - indicator_code : code de l'indicateur à récupérer
    - countries : liste des codes pays (codes ISO 2 lettres)
    - years : liste des années pour lesquelles récupérer les données
    - sexes : liste des sexes à récupérer ('SEX_MLE', 'SEX_FMLE', 'SEX_BTSX')

    Retourne :
    - pandas DataFrame avec les données.
    """
    base_url = "https://ghoapi.azureedge.net/api/"
    url = f"{base_url}{indicator_code}"

    # Préparer les filtres pour l'API
    filter_conditions = []

    if countries:
        country_conditions = ' or '.join([f"SpatialDim eq '{country}'" for country in countries])
        filter_conditions.append(f"({country_conditions})")

    if years:
        year_conditions = ' or '.join([f"TimeDim eq {year}" for year in years])
        filter_conditions.append(f"({year_conditions})")

    if sexes:
        sex_conditions = ' or '.join([f"Dim1 eq '{sex}'" for sex in sexes])
        filter_conditions.append(f"({sex_conditions})")

    params = {}
    if filter_conditions:
        filter_query = ' and '.join(filter_conditions)
        params['$filter'] = filter_query

    params['$top'] = 5000  # Ajustez ce nombre si nécessaire pour récupérer plus de données.

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(f"Erreur lors de la récupération des données : {response.status_code}")
        return None

    data = response.json()

    # Vérifier si les données contiennent les dimensions 'TimeDim' et 'Dim1'
    has_time_dim = any('TimeDim' in entry for entry in data.get('value', []))
    has_sex_dim = any('Dim1' in entry for entry in data.get('value', []))

    # Extraire les données
    data_rows = []

    if 'value' in data:
        for entry in data['value']:
            row = {
                'pays': entry.get('SpatialDim'),
                'annee': entry.get('TimeDim') if has_time_dim else None,
                'sexe': entry.get('Dim1') if has_sex_dim else None,
                'Value': entry.get('Value'),  # Valeur brute avec les bornes.
                'valeur': entry.get('NumericValue'),
                'borne_inferieure': entry.get('Low'),
                'borne_superieure': entry.get('High')
            }
            data_rows.append(row)

    else:
        print("Aucune donnée trouvée dans la réponse de l'API.")

    df = pd.DataFrame(data_rows)

    return df



In [31]:
df = load_metabolic_risk_data("NCD_BMI_30A")
df.head()


,pays,annee,sexe,Value,valeur,borne_inferieure,borne_superieure
0,BWA,2011,SEX_BTSX,14.4 [12.9-16.0],14.41709,12.88412,16.02893
1,LVA,1997,SEX_MLE,13.0 [10.3-15.9],12.99853,10.31592,15.89112
2,KHM,2006,SEX_MLE,1.1 [0.7-1.7],1.08977,0.66335,1.68620
3,DMA,2010,SEX_BTSX,23.3 [20.3-26.5],23.29871,20.28844,26.46879
4,AGO,2014,SEX_MLE,4.7 [2.4-7.9],4.66743,2.44310,7.92905


## Fonction spécifique à un jeu de données

## Préparer les données venant de site Global Burden of Disease (GBD) / Pas d'api ici

Le site Global Burden of Disease (GBD) est une plateforme en ligne qui fournit des données et des analyses sur la santé mondiale, en particulier sur la charge mondiale de morbidité (global burden of disease). Ce projet, géré par l'Institute for Health Metrics and Evaluation (IHME) de l'Université de Washington, a pour but de quantifier l'impact des maladies, des blessures et des facteurs de risque dans le monde entier, en mesurant leur effet sur la mortalité et la qualité de vie.


In [ ]:
# Evolution de chaque facteurs de risque
def load_evolution_facteurs(filepath="datas/evolution_chaque_facteurs/evolution_facteurs.csv"):
    import pandas as pd
    """Charge les données sur l'évolution des facteurs de risque."""
    return pd.read_csv(filepath, encoding="utf-8")  


In [104]:
data=load_evolution_facteurs()

In [105]:
data

,measure,location,sex,age,cause,rei,metric,year,val,upper,lower
0,Décès,Global,Homme,25 + ans,Toutes causes,"Eau, hygiène et assainissement",#,1990,5.575259e+05,9.127198e+05,312118.504077
1,Décès,Global,Femme,25 + ans,Toutes causes,"Eau, hygiène et assainissement",#,1990,5.903150e+05,9.953348e+05,317685.203782
2,Décès,Global,Les deux,25 + ans,Toutes causes,"Eau, hygiène et assainissement",#,1990,1.147841e+06,1.715821e+06,728883.867683
3,Décès,Global,Homme,25 + ans,Toutes causes,"Eau, hygiène et assainissement",%,1990,3.249086e-02,5.262791e-02,0.018058
4,Décès,Global,Femme,25 + ans,Toutes causes,"Eau, hygiène et assainissement",%,1990,3.976965e-02,6.721245e-02,0.021400
...,...,...,...,...,...,...,...,...,...,...,...
131643,Décès,Global,Femme,<70 ans,Toutes causes,Maltraitance de l’enfance,#,2021,2.187303e+03,4.713540e+03,345.270815
131644,Décès,Global,Les deux,<70 ans,Toutes causes,Maltraitance de l’enfance,#,2021,1.115561e+04,2.533907e+04,1935.625492
131645,Décès,Global,Homme,<70 ans,Toutes causes,Maltraitance de l’enfance,%,2021,4.450220e-04,1.017823e-03,0.000079
131646,Décès,Global,Femme,<70 ans,Toutes causes,Maltraitance de l’enfance,%,2021,1.711646e-04,3.765797e-04,0.000026
